参考
・langchainの基本的な流れ
https://note.com/unco3/n/nb8dc295db03b

・Chainの基本
https://qiita.com/taka_yayoi/items/f09678fe6dcd57c8d2b3

In [27]:
from config import *
import pandas as pd

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [29]:
# 文字起こしデータの読み込み
df_transcription = pd.read_csv(f"{PROCESSED_DIR}/sample_data.csv")[:10]

In [30]:
df_transcription

,start,end,transcript,speaker,confidence
0,0.0,0.1,では、来季の予算会議を始めます。各部署からの報告をお願いします。まずは佐藤さん、全体の見通し...,0,1
1,0.1,0.2,はい。来季の全体予算案につきまして、ご説明いたします。売上高は前年比5%増の850億円、営業...,1,1
2,0.2,0.3,なるほど。前年よりも成長を見込んでいるわけですね。各部門の詳細をお聞きしましょう。鈴木さん、...,0,1
3,0.3,0.4,はい。研究開発費は総額で95億円を計画しています。これは前年比8%増です。主な増加要因は、新...,2,1
4,0.4,0.5,8%増は大きいですね。具体的にどのような成果を期待していますか？,0,1
5,0.5,0.6,新規バイオ素材については、3年以内の製品化を目指しています。環境負荷低減については、主力製品...,2,1
6,0.6,0.7,その環境技術は、営業的にも大きなアピールポイントになりますね。,3,1
7,0.7,0.8,確かに。では高橋さん、営業部門の計画をお願いします。,0,1
8,0.8,0.9,はい。売上目標は佐藤さんの説明通り850億円です。内訳として、国内が500億円、海外が350...,3,1
9,0.9,1.0,アジア市場にはどのくらいの伸びを見込んでいますか？,0,1


In [31]:
# プロンプトテンプレートの作成
template = ChatPromptTemplate([
    ("system", "あなたは、WEB会議の質を高めるためのコンサルタントです。あなたは、userから与えられたWEB会議内の発言が結論ファーストであるかどうかを判定し、結論ファーストならば1、そうでなければ0として出力してください。\n{format_instructions}"),
    ("human", "{user_input}"),
])
# モデルの設定
model = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key= OPENAI_API_KEY)
# 出力形式の設定
first_conclusion_schema = ResponseSchema(name="conc_1st_flg", description="2値の結論ファーストフラグ。結論ファーストならば1、そうでなければ0とする。")
output_parser = StructuredOutputParser.from_response_schemas([first_conclusion_schema])
format_instructions = output_parser.get_format_instructions()

# チェーンの作成
chain = template | model | output_parser

# # チェーンの実行
# chain.invoke(
#     {
#         "format_instructions": format_instructions,
#         "user_input": "そうですね、アクセンチュアの方がレベルが高いとか、世間的にはいろんな意見があると思うのですが、自分は友達が多く所属していることもあり、PwCに行くことにしました。"
#     }
# )

In [41]:
results = []
for _, row in df_transcription.iterrows():
    text = row["transcript"]
    result = chain.invoke(
        {
            "format_instructions": format_instructions,
            "user_input": text
        }
    )
    results.append(result['conc_1st_flg'])

df_transcription["is_conclusion_1st"] = results

In [42]:
df_transcription

,start,end,transcript,speaker,confidence,is_conclusion_1st
0,0.0,0.1,では、来季の予算会議を始めます。各部署からの報告をお願いします。まずは佐藤さん、全体の見通し...,0,1,0
1,0.1,0.2,はい。来季の全体予算案につきまして、ご説明いたします。売上高は前年比5%増の850億円、営業...,1,1,1
2,0.2,0.3,なるほど。前年よりも成長を見込んでいるわけですね。各部門の詳細をお聞きしましょう。鈴木さん、...,0,1,0
3,0.3,0.4,はい。研究開発費は総額で95億円を計画しています。これは前年比8%増です。主な増加要因は、新...,2,1,1
4,0.4,0.5,8%増は大きいですね。具体的にどのような成果を期待していますか？,0,1,0
5,0.5,0.6,新規バイオ素材については、3年以内の製品化を目指しています。環境負荷低減については、主力製品...,2,1,1
6,0.6,0.7,その環境技術は、営業的にも大きなアピールポイントになりますね。,3,1,0
7,0.7,0.8,確かに。では高橋さん、営業部門の計画をお願いします。,0,1,0
8,0.8,0.9,はい。売上目標は佐藤さんの説明通り850億円です。内訳として、国内が500億円、海外が350...,3,1,1
9,0.9,1.0,アジア市場にはどのくらいの伸びを見込んでいますか？,0,1,0


In [43]:
df_transcription.to_csv(f"{RESULT_DIR}/result_sample_data.csv", index=False)